In [1]:
import argparse
from datetime import datetime
from pathlib import Path
from warnings import filterwarnings

import lightning.pytorch as pl
import pandas as pd
import torch
import yaml
from tqdm import tqdm

from dataset import TCGA_Program_Dataset
from datasets_manager import TCGA_Balanced_Datasets_Manager, TCGA_Datasets_Manager
from lit_models import LitFullModel
from model import Classifier, Feature_Extractor, Graph_And_Clinical_Feature_Extractor, Task_Classifier
from utils import config_add_subdict_key, get_logger, override_n_genes, set_random_seed, setup_logging
import shap


import matplotlib.pyplot as plt
import matplotlib
matplotlib.use('TkAgg')
import numpy as np
import seaborn as sns

SEED = 1126
set_random_seed(SEED)

IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html


In [2]:


def create_models_and_optimizers(config: dict):
    models: dict[str, torch.nn.Module] = {}
    optimizers: dict[str, torch.optim.Optimizer] = {}

    # Setup models. Do not use getattr() for better IDE support.
    for model_name, kargs in config['models'].items():
        if model_name == 'Graph_And_Clinical_Feature_Extractor':
            models['feat_ext'] = Graph_And_Clinical_Feature_Extractor(**kargs)
        elif model_name == 'Feature_Extractor':
            models['feat_ext'] = Feature_Extractor(**kargs)
        elif model_name == 'Task_Classifier':
            models['clf'] = Task_Classifier(**kargs)
        elif model_name == 'Classifier':
            models['clf'] = Classifier(**kargs)
        else:
            raise ValueError(f'Unknown model type: {model_name}')

    # Setup optimizers. If the key is 'all', the optimizer will be applied to all models.
    for key, optim_dict in config['optimizers'].items():
        opt_name = next(iter(optim_dict))
        if key == 'all':
            params = [param for model in models.values() for param in model.parameters()]
            optimizers[key] = getattr(torch.optim, opt_name)(params, **optim_dict[opt_name])
        else:
            optimizers[key] = getattr(torch.optim, opt_name)(models[key].parameters(), **optim_dict[opt_name])

    # Add models' structure to config for logging. TODO: Prettify.
    for model_name, torch_model in models.items():
        config[f'model.{model_name}'] = str(torch_model)
    return models, optimizers

In [7]:

# Select a config file.

config_path = '/home/zow/Multi-Cancer/config/light/MTL_TCGA.yaml'
config = config_path
with open(config, 'r') as f:
    config = yaml.load(f, Loader=yaml.FullLoader)
override_n_genes(config)                                                    # For multi-task graph models.
config_name =config_path.split('/')[-1].split('.')[0]

# Setup logging.
setup_logging(log_path := f'Logs/{config_name}/{datetime.now():%Y-%m-%dT%H:%M:%S}/')
logger = get_logger(config_name)
logger.info(f'Using Random Seed {SEED} for this experiment')
get_logger('lightning.pytorch.accelerators.cuda', log_level='WARNING')      # Disable cuda logging.
filterwarnings('ignore', r'.*Skipping val loop.*')                          # Disable val loop warning.

# Create dataset manager.
#here use torch lightning DS
data = {'TCGA_BLC': TCGA_Program_Dataset(**config['datasets'])}
if 'TCGA_Balanced_Datasets_Manager' == config['datasets_manager']['type']:
    manager = TCGA_Balanced_Datasets_Manager(datasets=data, config=config_add_subdict_key(config))
else:
    manager = TCGA_Datasets_Manager(datasets=data, config=config_add_subdict_key(config))

valid_results = []
# Cross validation.
for key, values in manager['TCGA_BLC']['dataloaders'].items():
    if isinstance(key, int) and config['cross_validation']:
        pass
    #     models, optimizers = create_models_and_optimizers(config)
    #     lit_model = LitFullModel(models, optimizers, config)
    #     trainer = pl.Trainer(                                               # Create sub-folders for each fold.
    #         default_root_dir=log_path,
    #         max_epochs=config['max_epochs'],
    #         log_every_n_steps=1,
    #         enable_model_summary=False,
    #         enable_checkpointing=False,
    #     )
    #     trainer.fit(lit_model, train_dataloaders=values['train'], val_dataloaders=values['valid'])
    #     trainer.test(lit_model, dataloaders=values['valid'], verbose=False) #verbose true prints the validation results for each fold
    #     # print validation results
        
    #     valid_results.append(trainer.test(lit_model, dataloaders=values['valid'], verbose=False)[0])
            
        
        
    elif key == 'train':
        train = values
    elif key == 'test':
        test = values



# Train the final model.
models, optimizers = create_models_and_optimizers(config)
lit_model = LitFullModel(models, optimizers, config)
trainer = pl.Trainer(
    default_root_dir=log_path,
    max_epochs=config['max_epochs'],
    enable_progress_bar=False,
    log_every_n_steps=1,
    logger=True,
)

[INFO]	Using Random Seed 1126 for this experiment
[INFO]	Creating a TCGA Program Dataset with 3 Projects...
